In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [3]:
spark = SparkSession.builder\
    .appName('austism-and-green-space')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2449170c-f7db-4114-8a32-f411d15d3895;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 571ms :: artifacts dl 28ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

22/08/06 01:44:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/06 01:44:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.parquet('s3a://projeto-pi-datalake-trusted/autism-greespace')

22/08/06 01:44:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
greenspace = df.withColumn("NonUrban_Land", sum([
        f.col("Water_Area"), f.col("Open_Land"), f.col("Forest_Land"), f.col("Barren_Land"), f.col("Grass_Land"), 
        f.col("Shrub_Land"), f.col("Agro_Land"), f.col("Wet_Land")]
))

In [6]:
greenspace.printSchema()

root
 |-- Autism: string (nullable = true)
 |-- Water_Area: string (nullable = true)
 |-- Open_Land: string (nullable = true)
 |-- Urban_Land: string (nullable = true)
 |-- Barren_Land: string (nullable = true)
 |-- Forest_Land: string (nullable = true)
 |-- Shrub_Land: string (nullable = true)
 |-- Grass_Land: string (nullable = true)
 |-- Agro_Land: string (nullable = true)
 |-- Wet_Land: string (nullable = true)
 |-- Pop: string (nullable = true)
 |-- Income_median: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- NonUrban_Land: double (nullable = true)



In [8]:
greenspace.write.mode('overwrite').partitionBy('year', 'month', 'day')\
    .csv('s3a://projeto-pi-datalake-analytics/autism-greespace', header=True)

22/08/06 01:44:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


22/08/06 01:44:30 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
